# Notebook for merging Earth Engine exports with FIA results.
This notebook takes remote sensing dataset summaries exported from Earth Engine and FIA Forest and Timberlands summaries and merges into a single table for downstream analysis.


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
VERSION = 'v5-1'

# Tabular results.
DRIVE_DIR = '/content/drive/MyDrive/LandSystem_RemoteSensing/Forest-Product-Comparison/datasets/'
FIA_FOREST_DIR = DRIVE_DIR + 'fia_forest_area/'
FIA_TIMBERLANDS_DIR = DRIVE_DIR + 'fia_timberland_area/'
EE_DIR = DRIVE_DIR + VERSION + '/'

# Output locations.
OUTPUT_DIR = DRIVE_DIR + VERSION + '_merged/'


In [5]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [6]:
# Merge TX-1 and TX-1 and write new TX csv.
tx_1 = pd.read_csv(EE_DIR + 'fpc_' + VERSION + '_TX-1.csv')
tx_2 = pd.read_csv(EE_DIR + 'fpc_' + VERSION + '_TX-2.csv')

tx = tx_1.add(tx_2)

tx.drop(columns=['dataset_id', 'state'], inplace=True)
tx.insert(0, 'dataset_id', tx_1['dataset_id'])
tx.insert(1, 'state', 'TX')
tx.set_index('dataset_id', inplace=True)

tx.to_csv(EE_DIR + 'fpc_' + VERSION + '_TX.csv')

In [7]:
import os

# Get file lists.
fia_forest_files = list(os.listdir(FIA_FOREST_DIR))
fia_timberlands_files = list(os.listdir(FIA_TIMBERLANDS_DIR))
ee_files = list(os.listdir(EE_DIR))

# Check lists for name formatting.
print(fia_forest_files)
print(fia_timberlands_files)
print(ee_files)

['VAR_forest_area_NV.csv', 'VAR_forest_area_ME.csv', 'VAR_forest_area_IL.csv', 'VAR_forest_area_CT.csv', 'VAR_forest_area_MO.csv', 'VAR_forest_area_NH.csv', 'VAR_forest_area_MS.csv', 'VAR_forest_area_MA.csv', 'VAR_forest_area_ND.csv', 'VAR_forest_area_OH.csv', 'VAR_forest_area_PA.csv', 'VAR_forest_area_LA.csv', 'VAR_forest_area_NE.csv', 'VAR_forest_area_TX.csv', 'VAR_forest_area_GA.csv', 'VAR_forest_area_OR.csv', 'VAR_forest_area_FL.csv', 'VAR_forest_area_IN.csv', 'VAR_forest_area_KS.csv', 'VAR_forest_area_SC.csv', 'VAR_forest_area_DE.csv', 'VAR_forest_area_NC.csv', 'VAR_forest_area_NY.csv', 'VAR_forest_area_TN.csv', 'VAR_forest_area_AR.csv', 'VAR_forest_area_RI.csv', 'VAR_forest_area_NM.csv', 'VAR_forest_area_MT.csv', 'VAR_forest_area_OK.csv', 'VAR_forest_area_KY.csv', 'VAR_forest_area_MI.csv', 'VAR_forest_area_CO.csv', 'VAR_forest_area_CA.csv', 'VAR_forest_area_MN.csv', 'VAR_forest_area_NJ.csv', 'VAR_forest_area_UT.csv', 'VAR_forest_area_SD.csv', 'VAR_forest_area_IA.csv', 'VAR_forest

In [8]:
def process_fia_results(
    dir: str,
    basename: str,
    product_id: str,
    state: str) -> pd.DataFrame:

  # Read in FIA results.
  fia_csv = pd.read_csv(f'{dir}{basename}_{state}.csv', index_col='YEAR')

  # Calculate FIA upper and lower bounds.
  fia_csv[product_id + '_upper'] = fia_csv['Area_Km2'] + fia_csv['StandDev_Km2']
  fia_csv[product_id + '_lower'] = fia_csv['Area_Km2'] - fia_csv['StandDev_Km2']

  # Clean up FIA table.
  fia_csv.rename(columns={'Area_Km2': product_id},
                 inplace=True)
  fia_csv.drop(columns=[
                        'Unnamed: 0',
                        'StandDev_Km2',
                        'Total_Variance',
                        'N_Forested'],
               inplace=True)

  # Transpose and set attributes.
  fia_csv_rows = fia_csv.transpose().reset_index()
  fia_csv_rows.rename(columns={'index': 'dataset_id'}, inplace=True)
  fia_csv_rows['state'] = state

  # Convert FIA column headers to strings to match EE results.
  fia_csv_rows.columns = fia_csv_rows.columns.astype(str)

  return fia_csv_rows

In [9]:
GHG_FIA_CSV = DRIVE_DIR + 'USFS_RDS-2021-0035/Data/CSV/LULUC_area.csv'

# Convert from thousands of hectares to km2
K_HA_TO_KM2 = 1000 * 0.01


def process_ghg_results(state: str, product_id: str) -> pd.DataFrame:
  ghg_fia = pd.read_csv(GHG_FIA_CSV)

  # Column with forest estimates.
  LULUC_COL = 'Land Use & Land Use Change Categories'

  ghg_fia['state'] = ghg_fia['State'].replace(us_state_to_abbrev)
  ghg_fia['dataset_id'] = product_id
  ghg_fia_forest = ghg_fia.loc[ghg_fia[LULUC_COL] == 'Total Forest Land'].copy()
  ghg_fia_forest.drop(columns=['State', LULUC_COL], inplace=True)

  ghg_fia_forest_select = ghg_fia_forest.loc[ghg_fia['state'] == state].copy()
  ghg_fia_forest_select.iloc[:, :-2] = ghg_fia_forest_select.iloc[:, :-2] * K_HA_TO_KM2

  return ghg_fia_forest_select


In [10]:
NRI_NONFEDERAL_CSV = DRIVE_DIR + 'nri_nonfederal_forest/nri_grazed_notgrazed_total_forest_acres.csv'

# Convert acres to km2
ACRE_TO_KM2 = (1 / 247.1)


def process_nri_results(state: str, product_id: str, col_value: str) -> pd.DataFrame:
  nri_nonfederal = pd.read_csv(NRI_NONFEDERAL_CSV)

  # Column with forest estimates.
  LULUC_COL = 'forest_type'

  nri_nonfederal['dataset_id'] = product_id
  nri_nonfederal = nri_nonfederal.loc[nri_nonfederal[LULUC_COL] == col_value].copy()
  nri_nonfederal.drop(columns=['forest_type', '1982'], inplace=True)

  nri_nonfederal_select = nri_nonfederal.loc[nri_nonfederal['state'] == state].copy()
  nri_nonfederal_select.iloc[:, 1:-1] = nri_nonfederal_select.iloc[:, 1:-1] * ACRE_TO_KM2

  return nri_nonfederal_select

In [11]:
def process_table(ee_file: str) -> pd.DataFrame:
  # Get state code from filename.
  state = ee_file.split('_')[2].split('.')[0]

  # Read in EE results.
  ee_csv = pd.read_csv(EE_DIR + ee_file)

  # Convert EE results from m2 to km2
  ee_csv.iloc[:, 2::] = ee_csv.iloc[:, 2::] / (1000 * 1000)

  forest = process_fia_results(
      FIA_FOREST_DIR, 'VAR_forest_area', 'FIA_forest', state)
  timberlands = process_fia_results(
      FIA_TIMBERLANDS_DIR, 'VAR_timberland_area', 'FIA_timberland', state)

  ghg_forests = process_ghg_results(state, 'EPA_forested')

  nri_total = process_nri_results(state, 'NRI_total', 'total')
  nri_grazed = process_nri_results(state, 'NRI_grazed', 'grazed')
  nri_notgrazed = process_nri_results(state, 'NRI_notgrazed', 'notgrazed')

  # Append FIA rows to EE table.
  final_csv = pd.concat(
      [ee_csv, forest, timberlands, ghg_forests, nri_total, nri_grazed, nri_notgrazed]).reset_index().drop(columns=['index'])
  final_csv.set_index('dataset_id', inplace=True)

  # Export final merged table to csv.
  name = VERSION + f'_merged_{state}'
  final_csv.to_csv(OUTPUT_DIR + name + '.csv')

  return final_csv.sort_index()

In [12]:
conus_df = pd.DataFrame()

# Loop over EE results.
for ee_file in ee_files:
  state = ee_file.split('_')[2].split('.')[0]

  # Check if we have a matching FIA file.
  if any(
      (state in f) for f in fia_forest_files) and any(
          (state in f) for f in fia_timberlands_files):
    print(f'Processing {state}')

    # Process and merge results files.
    table = process_table(ee_file)

    conus_df = pd.concat([conus_df, table]).groupby(level=0).sum(numeric_only=True)
    conus_df.replace(0, np.nan, inplace=True)

  else:
    print(f'No FIA results for this state: {state}')

print('Done!')

conus_df.to_csv(OUTPUT_DIR + 'v5-1_merged_CONUS.csv')

Processing IN
Processing KY
Processing AR
Processing WI
Processing IA
Processing RI


<ipython-input-10-7f0f853a0a34>:18: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  nri_nonfederal_select.iloc[:, 1:-1] = nri_nonfederal_select.iloc[:, 1:-1] * ACRE_TO_KM2


Processing DE
Processing CT
Processing VT
Processing NJ
Processing NH
Processing SC
Processing MD
Processing OH
Processing WV
Processing MS
Processing ME
Processing NY
Processing TN
Processing MO
Processing PA
Processing AL
Processing LA
Processing GA
Processing VA
Processing MA
Processing OK
Processing NC
Processing MI
Processing OR
Processing FL
Processing MN
Processing CO
Processing WA
Processing IL
Processing ND
Processing ID
Processing SD
Processing NM
Processing NE
Processing WY
Processing KS
No FIA results for this state: TX-2
Processing UT
No FIA results for this state: TX-1
Processing AZ
Processing NV
Processing MT
Processing CA
Processing TX
Done!
